<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/Clean_dataframe_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 6.2 MB/s eta 0:00:00


In [2]:
from bs4 import BeautifulSoup
import requests
import json
import unidecode
import pandas as pd
import numpy as np

import json
import re

import time
from tqdm import tqdm

In [3]:
!git clone 'https://github.com/mille055/Rec_Project.git'

Cloning into 'Rec_Project'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 206 (delta 41), reused 42 (delta 10), pack-reused 112
Receiving objects: 100% (206/206), 56.94 MiB | 23.99 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [15]:
import unidecode

def convert_si_to_number(x):
    total_stars = 0
    if 'K' in x:
        if len(x) > 1:
            total_stars = float(x.replace('K', '')) * 1000  # convert K to a thousand
    elif 'M' in x:
        if len(x) > 1:
            total_stars = float(x.replace('M', '')) * 1000000  # convert M to a million
    elif 'B' in x:
        total_stars = float(x.replace('B', '')) * 1000000000  # convert B to a Billion
    else:
        total_stars = int(x)  # Less than 1000
    return int(total_stars)


def clean_title(t):
    t = unidecode.unidecode(t)
    t = t.replace('\n', ' ')
    t = re.sub(r'[^\w\s]', '', t)
    t = re.sub(r'\d+', '', t)
    t = t.lower()
    t = t.strip()
    return t

def clean_description(d):
    d = unidecode.unidecode(d)
    d = d.replace('\n', ' ')
    d = re.sub(r'[^\w\s]', '', d)
    d = re.sub(r'\d+', '', d)
    if re.findall(r'(.*) brought to you by.*', d):
      d = re.sub(r'brought to you by.*', '', d)
    if re.search(r'(.*) sponsored by.*', d):
      d = re.sub(r'sponsored by.*', '', d)
    d = d.lower()
    d = d.strip()
    
    return d


def clean_description_list(dlist):
    
    new_string = ""
    for d in dlist:
      d = unidecode.unidecode(d)
      d = d.replace('\n', ' ')
      d = re.sub(r'[^\w\s]', '', d)
      d = re.sub(r'\d+', '', d)
      if re.findall(r'(.*) brought to you by.*', d):
        d = re.sub(r'brought to you by.*', '', d)
      if re.search(r'(.*) sponsored by.*', d):
        d = re.sub(r'sponsored by.*', '', d)
      d = d.lower()
      d = d.strip()
      new_string = new_string + " " + d
    return new_string


# Preferred method for cleaning the text currently
def clean_text(text):
    text = unidecode.unidecode(text)
    text = text.replace('\n', ' ')
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove "Sponsored by" phrases
    text = re.sub(r'(?i)sponsored\sby\s\w+', '', text)
    
    # Remove special characters and symbols (excluding spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    return text

# Define a function to join a list of strings
def join_strings(string_list, separator=" "):
    return separator.join(string_list)


def join_and_clean_text(string_list, separator=" "):
    # Join the list of strings
    single_text = join_strings(string_list, separator)

    # Clean the combined text
    cleaned_text = clean_text(single_text)

    return cleaned_text

In [7]:
myscraped = pd.read_pickle('/content/Rec_Project/data/podcast_df_040423.pkl')

In [8]:
### need to clean up the episode_description column
example_to_clean = myscraped.iloc[0].episode_descriptions

In [9]:
example_block = join_strings(example_to_clean)
clean_example = clean_text(example_block)
clean_example

'In celebration of our new premium formattwo premium episodes a month for both Apple Podcast premium subscribers and Patreon subscribers starting April 2023we are sharing this premium episode with all listeners  The legend of Kentuckys Pope Lick Monster is inexorably tied to the train bridge it supposedly guardsbut what is the real danger the goatsheep cryptid or our human need to seek out danger   Hosted by Laurah Norton  Written by Liv Fallon  Researched by Jessica Lee   Produced and Scriptedited by Maura Currie  Engineered and Scored by Chaes Gray   Preorder Laurahs book LAY THEM TO REST   Butcherbox Get free Chicken Nuggets for a Year and 10 percent off your first box when you sign up today Thats a 22 oz bag of glutenfree chicken nuggets in every order for a year when you sign up at butcherboxcomOST and use code OST  Join us on Patreon for early release and adfree episodes exclusive stories and bonus episodes  Find us on Twitter  Instagram   and Facebook  Interested in advertising 

In [17]:
myscraped_cleaned = myscraped.copy()
myscraped_cleaned.episode_descriptions = myscraped_cleaned.episode_descriptions.apply(join_and_clean_text)
myscraped_cleaned

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id,rating,user
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,In celebration of our new premium formattwo pr...,1526579247,5,RobinFerris
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,In celebration of our new premium formattwo pr...,1526579247,1,Pops.99
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,In celebration of our new premium formattwo pr...,1526579247,5,ReddEye81
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,In celebration of our new premium formattwo pr...,1526579247,2,Keyta7777
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,In celebration of our new premium formattwo pr...,1526579247,4,Okkupent
...,...,...,...,...,...,...,...,...,...,...,...,...
657194,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,Hey Angels In todays episode of the Quality Qu...,1512702672,5,Monijansand
657195,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,Hey Angels In todays episode of the Quality Qu...,1512702672,5,trinityangel13
657196,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,Hey Angels In todays episode of the Quality Qu...,1512702672,5,Kweenkeys
657197,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,Hey Angels In todays episode of the Quality Qu...,1512702672,5,JoyfulJoyfulWOG


In [18]:
myscraped_cleaned.to_pickle('podcast_clean_df_040423.pkl')